In [ ]:
import ccxt
import pandas as pd
import datetime
import time
from typing import List, Dict, Any

In [ ]:
def setup_exchange(exchange_id: str = 'binance', api_key: str = None, api_secret: str = None):

    exchange_class = getattr(ccxt, exchange_id)
    exchange_config = {
        'enableRateLimit': True,  
    }
    
    if api_key and api_secret:
        exchange_config.update({
            'apiKey': api_key,
            'secret': api_secret
        })
    
    exchange = exchange_class(exchange_config)
    return exchange

def get_market_id(exchange, base_currency: str = 'BTC', quote_currency: str = 'USDC', contract_type: str = 'perpetual'):

    exchange.load_markets()
    
    for symbol, market in exchange.markets.items():
        is_matching_pair = (
            market['base'] == base_currency and 
            market['quote'] == quote_currency and
            market.get('linear', False) and  # Linear contracts settle in the quote currency
            market.get('future', False) and  # Is a futures contract
            market.get('swap', False)  # Is a perpetual swap
        )
        
        if is_matching_pair:
            return symbol
    
    raise ValueError(f"Could not find {base_currency}/{quote_currency} perpetual pair on {exchange.id}")

def fetch_historical_trades(exchange, symbol: str, since: int = None, limit: int = 1000, params: dict = None) -> List[Dict]:

    all_trades = []
    params = params or {}
    
    if not exchange.has['fetchTrades']:
        raise Exception(f"{exchange.id} does not support fetching trades")
    
    if exchange.id == 'binance':
        if 'future' in params or symbol.endswith(':USDC'):
            params['category'] = 'linear'
    
    try:
        trades = exchange.fetch_trades(symbol, since, limit, params)
        all_trades.extend(trades)
        return all_trades
    except Exception as e:
        print(f"Error fetching trades: {str(e)}")
        return []

def fetch_all_historical_trades(exchange, symbol: str, start_time: datetime.datetime, 
                               end_time: datetime.datetime = None, batch_size: int = 1000) -> List[Dict]:

    if end_time is None:
        end_time = datetime.datetime.now()
    
    since = int(start_time.timestamp() * 1000)
    until = int(end_time.timestamp() * 1000)
    
    all_trades = []
    while since < until:
        print(f"Fetching trades since {datetime.datetime.fromtimestamp(since/1000)}")
        
        trades = fetch_historical_trades(exchange, symbol, since, batch_size)
        
        if not trades:
            print("No more trades found or rate limit reached")
            break
            
        all_trades.extend(trades)
        
        if trades:
            last_trade_time = trades[-1]['timestamp'] + 1
            since = last_trade_time
        else:

            since += 86400000 
        
        time.sleep(exchange.rateLimit / 1000)
    
    return all_trades

def trades_to_dataframe(trades: List[Dict]) -> pd.DataFrame:

    df = pd.DataFrame(trades)
    
    if 'price' not in df.columns and 'info' in df.columns:
        df['price'] = df['info'].apply(lambda x: float(x.get('price', 0)))
        
    if 'amount' not in df.columns and 'info' in df.columns:
        df['amount'] = df['info'].apply(lambda x: float(x.get('qty', 0) or x.get('amount', 0)))
    
    if 'timestamp' in df.columns:
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    if 'info' in df.columns:
        df.drop('info', axis=1, inplace=True)
    
    return df

def download_tick_data(
    base_currency: str = 'BTC', 
    quote_currency: str = 'USDC', 
    days_back: int = 2,
    exchange_id: str = 'binance',
    api_key: str = None,
    api_secret: str = None,
    output_filename: str = None
) -> pd.DataFrame:

    exchange = setup_exchange(exchange_id, api_key, api_secret)

    try:
        symbol = get_market_id(exchange, base_currency, quote_currency)
        print(f"Found market: {symbol}")
    except ValueError as e:
        print(f"Error: {str(e)}")

        symbol = f"{base_currency}/{quote_currency}:USDC"
        print(f"Trying fallback symbol: {symbol}")
    

    end_time = datetime.datetime.now()
    start_time = end_time - datetime.timedelta(days=days_back)
    
    print(f"Fetching trades for {symbol} from {start_time} to {end_time}")
    
    trades = fetch_all_historical_trades(exchange, symbol, start_time, end_time)
    print(f"Fetched {len(trades)} trades")
    
    trades_df = trades_to_dataframe(trades)
    
    if output_filename:
        trades_df.to_csv(output_filename, index=False)
        print(f"Tick data saved to {output_filename}")
    else:
        default_filename = f"{exchange_id}_{base_currency.lower()}{quote_currency.lower()}_trades.csv"
        trades_df.to_csv(default_filename, index=False)
        print(f"Tick data saved to {default_filename}")
    
    return trades_df

def convert_ticks_to_candles(
    tick_data: pd.DataFrame = None,
    tick_size: int = 100,
    input_filename: str = None,
    output_filename: str = None
) -> pd.DataFrame:

    if tick_data is None:
        if input_filename is None:
            raise ValueError("Either tick_data or input_filename must be provided")
        
        print(f"Loading tick data from {input_filename}")
        tick_data = pd.read_csv(input_filename)
        
        if 'datetime' not in tick_data.columns and 'timestamp' in tick_data.columns:
            tick_data['datetime'] = pd.to_datetime(tick_data['timestamp'], unit='ms')
    
    if 'timestamp' in tick_data.columns:
        tick_data = tick_data.sort_values('timestamp')
    else:
        tick_data = tick_data.sort_values('datetime')
    

    tick_data['group'] = tick_data.index // tick_size
    
    candles = tick_data.groupby('group').agg({
        'price': ['first', 'max', 'min', 'last'],  # OHLC
        'amount': 'sum',  # Volume
        'timestamp': 'first'  # Timestamp
    })
    
    candles.columns = ['open', 'high', 'low', 'close', 'volume', 'timestamp']
    
    candles['datetime'] = pd.to_datetime(candles['timestamp'], unit='ms')
    
    if output_filename:
        candles.to_csv(output_filename, index=False)
        print(f"Candle data saved to {output_filename}")
    elif input_filename:
        default_filename = input_filename.replace('_trades.csv', f'_tick{tick_size}_candles.csv')
        candles.to_csv(default_filename, index=False)
        print(f"Candle data saved to {default_filename}")
    
    return candles

# Example usage
if __name__ == "__main__":
    # Example 1: Download tick data and convert to candles in one go
    # ticks = download_tick_data(
    #     base_currency='BTC', 
    #     quote_currency='USDC', 
    #     days_back=2, 
    #     exchange_id='binance'
    # )
    
    # candles = convert_ticks_to_candles(
    #     tick_data=ticks,
    #     tick_size=100
    # )
    
    # Example 2: Just download tick data
    # download_tick_data(
    #     base_currency='BTC',
    #     quote_currency='USDC',
    #     days_back=9,
    #     output_filename='btcusdc_ticks.csv'
    # )
    
    # Example 3: Just convert existing tick data to candles
    # convert_ticks_to_candles(
    #     input_filename='/Users/sushanth/Desktop/Nano_scalping/binance_btcusdc_trades.csv',
    #     tick_size=100,
    #     output_filename='btcusdc_30tick_candles.csv'
    # )